<h3>Notebook that will go through a handful of analyses step by step instead of all at once as in total_model_analysis.py

In [1]:
import numpy as np
import os
import random
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import Image
from transvae import trans_models
from transvae.transformer_models import TransVAE
from transvae.rnn_models import RNN, RNNAttn
from transvae.wae_models import WAE
from transvae.aae_models import AAE
from transvae.tvae_util import *
from transvae import analysis

e:\Anaconda\envs\amp21\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [2]:
%config Completer.use_jedi = False
%load_ext autoreload
%autoreload 2

import IPython.display as Disp
np.set_printoptions(suppress=True)

This tutorial will demonstrate how to visualize and interpret model memory and how to evaluate model performance on a number of metrics. Full scripts for training models, generating samples and calculating attention weights are provided and instructions on how to use those scripts are included in the README. The functions demonstrated in this tutorial do not have pre-written high throughput scripts but can be used on larger input sizes if desired. 

# Model Reconstruction Performance

A set of metrics on reconstruction accuracy of the different models is presented below. Some parameters need to be selected:
<ul>
    <li>data size: int --how many samples from the data to laod
    <li>data selection: string  --training, testing, full_no_shuffle
    <li>model_src: string --path to model checkpoint
    <li>models : RNN, WAE, AAE, RNNAttn, TransVAE --model selectiong from listed
</ul>

In [15]:
num_sequences =100
data_selection = "data"
model_src = "checkpointz/amp_aae//sunistarv4_emb128_latent128//300_aae-128_peptide.ckpt"
#if loss log files are present you can uncomment and run the cell below to plots loss curves
#src = 'checkpointz//amp_aae//sunistarv4_emb128_latent128//log_aae-128_peptide.txt' #src of the loss output file
model = AAE(load_fn=model_src)
#manual override of HARDWARE device for CKPT loading
model = AAE()
model.params['HARDWARE']= 'cpu'
model.load(checkpoint_path=model_src)
print('Are we using cuda?: ',next(model.model.parameters()).is_cuda)

save_dir= "model_analyses//"+model.name+"test" #each model will have its own directory
if not os.path.exists(save_dir):os.mkdir(save_dir) 
save_dir= save_dir+"//" #actually enter the folder that was created above

save_df = pd.DataFrame() #this will hold the number variables and save to CSV

gpu = False

if "data" in data_selection:
    data = pd.read_csv('data//peptide_test.txt').to_numpy() 
data_1D = data[:num_sequences,0] #gets rid of extra dimension

Are we using cuda?:  False


In [5]:
# Needs :"src" above with log files from training "not on github repo"
# tot_loss = analysis.plot_loss_by_type(src,loss_types=['tot_loss'])
# plt.savefig(save_dir+'tot_loss.png',dpi=200)
# recon_loss = analysis.plot_loss_by_type(src,loss_types=['recon_loss'])
# plt.savefig(save_dir+'recon_loss.png',dpi=200)
# kld_loss = analysis.plot_loss_by_type(src,loss_types=['kld_loss'])

# plt.savefig(save_dir+'kld_loss.png',dpi=200)
# prob_bce_loss = analysis.plot_loss_by_type(src,loss_types=['prop_bce_loss'])
# plt.savefig(save_dir+'prob_bce_loss.png',dpi=200)
# if 'aae' in src:
#     disc_loss = analysis.plot_loss_by_type(src,loss_types=['disc_loss'])
#     plt.savefig(save_dir+'disc_loss.png',dpi=200)
# if 'wae' in src:
#     mmd_loss = analysis.plot_loss_by_type(src,loss_types=['mmd_loss'])
#     plt.savefig(save_dir+'mmd_loss.png',dpi=200)

In [16]:
LOAD=False
if LOAD: #this allows loading of reconstructed sequences from a file to save time
    recon_src = 'slurm_analyses//rnn-128_peptide_latent_32//saved_info.csv'
    recon_df = pd.read_csv(recon_src)
    reconstructed_seq = recon_df['reconstructions'].to_list()[:num_sequences]
    props = torch.Tensor(recon_df['predicted properties'][:num_sequences])
else:
    model.params['BATCH_SIZE'] = 100 #batch size must match total size of input data
    reconstructed_seq, props = model.reconstruct(data[:num_sequences], log=False, return_mems=False)
    for og_token, reconstructed_token in zip(data_1D, reconstructed_seq):
        print('{} <- Original'.format(og_token))
        print('{} <- Reconstruction'.format(reconstructed_token))
        print('\n')

if gpu:
    torch.cuda.empty_cache() #free allocated CUDA memory

cpu
decoding sequences of max length  125 current position:  0
decoding sequences of max length  125 current position:  100
MQQANFCEYIRELREEAKMPLRKLAAIMDIDQGTLSKLE <- Original
MQQANFCEYIRELREEAKMPLRKLAAIMDIDQGTLSKLE <- Reconstruction


MIFSNKWFAAKLFLKTTDFLIFMVKLLPEDEQSQNLKSTFCNYLS <- Original
MIFSNKWAFAFLKKLTTDFLIKMVLFLPEQDESELSTKFNCQFLYR <- Reconstruction


NQQLLGAM <- Original
NQQLLGAM <- Reconstruction


CDKVPAVPWMRRSCRIPPRPRTGRGNGRGLRCAGGRPPAARWSL <- Original
CDAVKPVPMCGRPPCSRRIPRPTGGRNRGGLCRGARPPAGWALSR <- Reconstruction


MPKKTIIKARDEQIQAMKEKKQAIERLREAAIT <- Original
MPKKTIIKARDEQIQAMKEKKQAIERLREAAIT <- Reconstruction


MAGRTALSALINKLMDRSKTVFFRHFNLLISLQSLSHWLYRTHVK <- Original
MAGRTALSALINKLMDRSKTVFFRHFNLLISLQSLHSLYWRTHVK <- Reconstruction


PPCIVDPHPTPMGPID <- Original
PPCIVDPHPTPMGPID <- Reconstruction


CDFIF <- Original
CDIFF <- Reconstruction


MANSDSIDNNNQGSMYNKDKTVTTNQRRF <- Original
MANSDSIDNNNQGSMYNKDKTVTTNQRRF <- Reconstruction


MLSFYVDTANISASYVDNSNQTQVFPLTYQGVAITFITVA

In [ ]:
#Experimental section using reconstructions from previous run to save time on analyses

# training = pd.read_csv('notebooks//example_data//train_test//peptide_train.txt').to_numpy()
# train_idx_list = [np.where(data==training[idx][0]) for idx in range(len(training))]

# testing = pd.read_csv('notebooks//example_data//train_test//peptide_test.txt').to_numpy()
# test_idx_list = [np.where(data==testing[idx][0]) for idx in range(len(testing))]

# test=True
# train=False
# if test:
#     batch_recon_len = len(reconstructed_seq)
#     reconstructed_seq = [reconstructed_seq[test_idx_list[i][0][0]] for i in range(len(test_idx_list)) if test_idx_list[i][0][0]<batch_recon_len]
#     data_1D= [data_1D[test_idx_list[i][0][0]] for i in range(len(test_idx_list)) if test_idx_list[i][0][0]<batch_recon_len]
#     props = [props[test_idx_list[i][0][0]] for i in range(len(test_idx_list)) if test_idx_list[i][0][0]<batch_recon_len]
#     props=torch.Tensor(props)
#     data = testing[:][0]
#     true_props_data = pd.read_csv('notebooks//example_data//function_full_no_shuff.txt').to_numpy()
#     true_props = true_props_data[0:num_sequences,0]
#     true_props= [true_props[test_idx_list[i][0][0]] for i in range(len(test_idx_list)) if test_idx_list[i][0][0]<batch_recon_len]
# if train:
#     batch_recon_len = len(reconstructed_seq)
#     reconstructed_seq = [reconstructed_seq[train_idx_list[i][0][0]] for i in range(len(train_idx_list)) if train_idx_list[i][0][0]<batch_recon_len]
#     data_1D= [data_1D[train_idx_list[i][0][0]] for i in range(len(train_idx_list)) if train_idx_list[i][0][0]<batch_recon_len]
#     props = [props[train_idx_list[i][0][0]] for i in range(len(train_idx_list)) if train_idx_list[i][0][0]<batch_recon_len]
#     props=torch.Tensor(props)
#     data = training[:][0]
#     true_props_data = pd.read_csv('notebooks//example_data//function_full_no_shuff.txt').to_numpy()
#     true_props = true_props_data[0:num_sequences,0]
#     true_props= [true_props[train_idx_list[i][0][0]] for i in range(len(train_idx_list)) if train_idx_list[i][0][0]<batch_recon_len]

# n_data_1D=[]
# n_reconstructed_seq=[]
# for idx,seq in enumerate(data_1D):
#     if len(seq)<50:
#         n_data_1D.append(seq)
#         n_reconstructed_seq.append(reconstructed_seq[idx])
# data_1D=n_data_1D
# reconstructed_seq=n_reconstructed_seq

In [ ]:
save_df['reconstructions'] = reconstructed_seq #placing the saves on a line separate from the ops allows for editing
save_df['predicted properties'] = [prop.item() for prop in props[:len(reconstructed_seq)]]

<ul>MCC info:
    <li>+1 represents a perfect prediction
    <li>0 no better than random prediction
    <li>−1 indicates total disagreement between prediction and observation.
</ul>

In [ ]:
true_props_data = pd.read_csv('data//function_test.txt').to_numpy()
true_props = true_props_data[0:num_sequences,0]
prop_acc, prop_conf, MCC=calc_property_accuracies(props[:len(reconstructed_seq)],true_props[:len(reconstructed_seq)])

In [ ]:
save_df['property prediction accuracy'] = prop_acc
save_df['property prediction confidence'] = prop_conf
save_df['MCC'] = MCC

Token accuracies are accuracies per token, 
<ul>
    <li>sequence accuracies are accuracies per sequence
    <li>token accuracies are accuracies for each token averaged over all tokens in input dataset
    <li>position accuracies are per sequence position

In [ ]:
# First we tokenize the input and reconstructed smiles
input_sequences = []
for seq in data_1D:
    input_sequences.append(peptide_tokenizer(seq))
output_sequences = []
for seq in reconstructed_seq:
    output_sequences.append(peptide_tokenizer(seq))

In [ ]:
seq_accs, tok_accs, pos_accs, seq_conf, tok_conf, pos_conf  = calc_reconstruction_accuracies(input_sequences, output_sequences)

In [ ]:
save_df['sequence accuracy'] = seq_accs
save_df['sequence confidence'] = seq_conf
save_df['token accuracy'] = tok_accs
save_df['token confidence'] = tok_conf

save_df = pd.concat([pd.DataFrame({'position_accs':pos_accs,'position_confidence':pos_conf }), save_df], axis=1)

Plotting the accuracy on token position

In [ ]:
plt.plot(pos_accs)
#plt.plot(pos_accs_pep)
plt.xlabel('Sequence Position')
plt.ylabel('Accuracy')
plt.savefig(save_dir+'token_position_accuracy.png')
plt.show()

# Visualizing Model Memory

The memory of a model is analogous to the probability distribution of molecular embeddings that it has learned during training. A single molecular embedding is the size 128 vector at the center of the variational bottleneck. Each model has a built-in method for calculating and returning the model memory for a set of input structures, `calc_mems()`. ***(note - we plot the mean vector rather than the reparameterized vector so we can identify and analyze the meaningful latent dimensions)***

In [ ]:
if model.model_type =='aae':
    mus, _, _ = model.calc_mems(data[:50_000], log=False, save=False) 
elif model.model_type == 'wae':
    mus, _, _ = model.calc_mems(data[:50_000], log=False, save=False) 
else:
    mems, mus, logvars = model.calc_mems(data[:], log=False, save=False) #subset size 1200*35=42000 would be ok

Shannon information entropy

In [ ]:
from transvae.tvae_util import *

In [ ]:
#save the list of entropies for each latent dim
vae_entropy_mus = calc_entropy(mus)
save_df = pd.concat([save_df,pd.DataFrame({'mu_entropies':vae_entropy_mus})], axis=1)
if model.model_type != 'wae' and model.model_type!= 'aae': #these don't have a variational type bottleneck
    vae_entropy_mems  = calc_entropy(mems)
    save_df = pd.concat([save_df,pd.DataFrame({'mem_entropies':vae_entropy_mems})], axis=1)
    vae_entropy_logvars = calc_entropy(logvars)
    save_df = pd.concat([save_df,pd.DataFrame({'logvar_entropies':vae_entropy_logvars})], axis=1)

In [ ]:
fig = plt.figure(figsize=(6,3))

plt.bar(range(len(vae_entropy_mus)), vae_entropy_mus)
plt.xlabel('Latent Dimension')
plt.ylabel('Entropy (bits)')
plt.savefig(save_dir+'mem_entropy')

In [ ]:
if model.model_type != 'wae' and model.model_type!= 'aae':
    fig = plt.figure(figsize=(6,3))
    plt.bar(range(len(vae_entropy_mems)), vae_entropy_mems)
    plt.xlabel('Latent Dimension')
    plt.ylabel('Entropy (bits)')
    plt.savefig(save_dir+'mu_entropy')

In [ ]:
if model.model_type != 'wae' and model.model_type!= 'aae':
    fig = plt.figure(figsize=(6,3))
    plt.bar(range(len(vae_entropy_logvars)), vae_entropy_logvars)
    plt.xlabel('Latent Dimension')
    plt.ylabel('Entropy (bits)')
    plt.savefig(save_dir+'logvar_entropy')

We can see that some dimensions have significantly more information contained across the 25 samples than others and they correspond with the selective memory visualization shown above. We can sum the entropy of all dimensions to find the full model entropy. Again, note that we would need a larger sample size to converge the model entropy.

In [ ]:
fig = plt.figure(figsize=(20,10))
plt.imshow(mus[:100], vmin=-6, vmax=6, aspect='auto', cmap='seismic')
plt.rc('font', size=10)          # controls default text sizes
plt.rc('axes', titlesize=40)     # fontsize of the axes title
plt.rc('axes', labelsize=30)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=10)    # fontsize of the tick labels
plt.rc('ytick', labelsize=10)    # fontsize of the tick labels
plt.xticks()
plt.yticks()
plt.ylabel("Embedded peptides", )
plt.xlabel("Latent dimensions")
plt.title('Latent Embedding of a 2000 Epoch RNN Model')
plt.colorbar(fraction=0.046, pad=0.04)
plt.savefig(save_dir+'mus.png', dpi=200, transparency=False)
plt.show()

<h4>Evaluate the trustworthiness of the mapping from raw input data to latent space manifold

In [ ]:
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.manifold import trustworthiness
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import plotly.express as px

true_prop_src = 'data\\function_test.txt' #if property predictor load the true labels
#create random index and re-index ordered memory list creating n random sub-lists (ideally resulting in IID random lists)
random_idx = np.random.permutation(np.arange(stop=mus.shape[0]))
mus = mus[random_idx]
data = data[random_idx]

subsample_start=0
subsample_length=mus.shape[0] #this may change depending on batch size

#(for length based coloring): record all peptide lengths iterating through input
pep_lengths = []
for idx, pep in enumerate(data[subsample_start:(subsample_start+subsample_length)]):
    pep_lengths.append( len(pep[0]) )   
#(for function based coloring): pull function from csv with peptide functions
s_to_f =pd.read_csv(true_prop_src)    
function = s_to_f['peptides'][subsample_start:(subsample_start+subsample_length)]
function = function[random_idx] #account for random permutation

pca = PCA(n_components=5)
pca_batch =pca.fit_transform(X=mus[:])

#plot format dictionnaries
titles={'text':'{}'.format(model.model_type.replace("_"," ").upper()),
                      'x':0.5,'xanchor':'center','yanchor':'top','font_size':40}
general_fonts={'family':"Helvetica",'size':30,'color':"Black"}
colorbar_fmt={'title_font_size':30,'thickness':15,'ticks':'','title_text':'Lengths',
                           'ticklabelposition':"outside bottom"}

fig = px.scatter(pd.DataFrame({"PC1":pca_batch[:,0],"PC2":pca_batch[:,1], "lengths":pep_lengths}),
            symbol_sequence=['hexagon2'],x='PC1', y='PC2', color="lengths",
            color_continuous_scale='Jet',template='simple_white', opacity=0.9)
fig.update_traces(marker=dict(size=9))
fig.update_layout(title=titles,xaxis_title="PC1", yaxis_title="PC2",font=general_fonts)
fig.update_coloraxes(colorbar=colorbar_fmt)
fig.write_image(save_dir+'pca_length.png', width=900, height=600)

fig = px.scatter(pd.DataFrame({"PC1":pca_batch[:,0],"PC2":pca_batch[:,1], 
                                "Function":list(map(lambda itm: "AMP" if itm==1 else "NON-AMP",function))}),
                                x='PC1', y='PC2', color="Function",symbol_sequence=['x-thin-open','circle'],
                                template='simple_white',symbol='Function', opacity=0.8) 
fig.update_traces(marker=dict(size=9))
fig.update_layout(title=titles,xaxis_title="PC1",yaxis_title="PC2",font=general_fonts)
fig.write_image(save_dir+'pca_function.png', width=900, height=600)
# #create n subsamples and calculate silhouette score for each
# latent_mem_func_subsamples = []
# pca_func_subsamples = []
# n=20
# for s in range(n):
#     s_len = len(mus)//n #sample lengths
#     mem_func_sil = metrics.silhouette_score(mus[s_len*s:s_len*(s+1)], function[s_len*s:s_len*(s+1)], metric='euclidean')
#     latent_mem_func_subsamples.append(mem_func_sil)
#     XY = [i for i in zip(pca_batch[s_len*s:s_len*(s+1),0], pca_batch[s_len*s:s_len*(s+1),1])]
#     pca_func_sil = metrics.silhouette_score(XY, function[s_len*s:s_len*(s+1)], metric='euclidean')
#     pca_func_subsamples.append(pca_func_sil)
# save_df = pd.concat([save_df,pd.DataFrame({'latent_mem_func_silhouette':latent_mem_func_subsamples})], axis=1)
# save_df = pd.concat([save_df,pd.DataFrame({'pca_func_silhouette':pca_func_subsamples})], axis=1)

# Plot the explained variances
features = range(pca.n_components_)
plt.bar(features, pca.explained_variance_ratio_*100, color='black')
plt.xlabel('PCA features')
plt.ylabel('variance %')
plt.xticks(features)
plt.savefig(save_dir+'variance_explained.png')

<h4>Trustworthiness, Continuity, Steadiness, Cohesiveness </h4>

In [ ]:
import coranking #coranking.readthedocs.io
from coranking.metrics import trustworthiness, continuity, LCMC
from transvae.snc import SNC #github.com/hj-n/steadiness-cohesiveness

trust_subsamples = []
cont_subsamples = []
lcmc_subsamples = []
steadiness_subsamples = []
cohesiveness_subsamples = []

n=35
parameter = { "k": 50,"alpha": 0.1 } #for steadiness and cohesiveness
for s in range(n):
    s_len = len(mus)//n #sample lengths
    Q = coranking.coranking_matrix(mus[s_len*s:s_len*(s+1)], pca_batch[s_len*s:s_len*(s+1)])
    trust_subsamples.append( np.mean(trustworthiness(Q, min_k=1, max_k=50)) )
    cont_subsamples.append( np.mean(continuity(Q, min_k=1, max_k=50)) )
    lcmc_subsamples.append( np.mean(LCMC(Q, min_k=1, max_k=50)) )
    print(n,trust_subsamples[s],cont_subsamples[s],lcmc_subsamples[s])
    
    metrics = SNC(raw=mus[s_len*s:s_len*(s+1)], emb=pca_batch[s_len*s:s_len*(s+1)], iteration=300, dist_parameter=parameter)
    metrics.fit() #solve for steadiness and cohesiveness
    steadiness_subsamples.append(metrics.steadiness())
    cohesiveness_subsamples.append(metrics.cohesiveness())
    print(metrics.steadiness(),metrics.cohesiveness())
    Q=0 #trying to free RAM
    metrics=0
    torch.cuda.empty_cache() #free allocated CUDA memory
    
save_df = pd.concat([save_df,pd.DataFrame({'latent_to_PCA_trustworthiness':trust_subsamples})], axis=1)
save_df = pd.concat([save_df,pd.DataFrame({'latent_to_PCA_continuity':cont_subsamples})], axis=1)
save_df = pd.concat([save_df,pd.DataFrame({'latent_to_PCA_lcmc':lcmc_subsamples})], axis=1)
save_df = pd.concat([save_df,pd.DataFrame({'latent_to_PCA_steadiness':steadiness_subsamples})], axis=1)
save_df = pd.concat([save_df,pd.DataFrame({'latent_to_PCA_cohesiveness':cohesiveness_subsamples})], axis=1)

In [ ]:
save_df.to_csv(save_dir+"saved_info.csv", index=False)